In [292]:
from bs4 import BeautifulSoup
import asyncio
import time
import threading
from aiohttp import ClientSession
import re

In [67]:
async def get(url):
  async with ClientSession() as session:
    async with session.get(url) as response:
      return await response.read()

In [163]:
loop = asyncio.get_event_loop()
res = loop.run_until_complete(get('http://profiles.asee.org/profiles/8191/print_all'))
soup = BeautifulSoup(res, 'html.parser')

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [144]:
colleges = {'1':['8191'],'2':['8192']}
'''
{ 'WUSTL': ['2019id', '2018id', ... ]}

'''

url = 'http://profiles.asee.org/profiles/{}/print_all
fetch = []
for key, value in colleges.items():
  print(value)
  fetch.append(get(url.format(value[0])))
  
res = loop.run_until_complete(asyncio.gather(*fetch))

['8191']
['8192']


In [164]:
name = soup.find('h1').text[:-7]

In [165]:
institution_type = soup.find('h3', text='General Information').find_next_sibling('table').find_all('td')[1].text

In [229]:
undergrad_enroll = soup.find('p', class_='highlighted', text='Total Enrollment').find_next_sibling('table').find_all('td')[1].text.replace(',','')

In [230]:
graduate_enroll = soup.find('p', class_='highlighted', text='Total Enrollment').find_next_sibling('table').find_all('td')[3].text.replace(',','')

In [222]:
description = soup.find('p', class_='highlighted', text='Engineering College Description and Special Characteristics').find_next_sibling('p').text

In [ ]:
#bach_programs_table = soup.find('p', class_='highlighted', text='Bachelor\'s Degree Program(s)').find_next_sibling('table')


In [236]:
teaching_tenure_table = soup.find('p', class_='highlighted', text='Teaching, Tenure-Track: Full Professor Profiles').find_next_sibling('table')
teaching_tenure = parse_table(teaching_tenure_table)

In [247]:
tuition_list = parse_table(soup.find('h3', text='Expenses & Financial Aid').find_next_sibling('table'))[1]
tuition = tuition_list[1:3] if len(tuition_list) > 2 else tuition_list[1]
tuition = [t.replace('$','').replace(',','') for t in tuition]

In [258]:
test_scores = soup.find('p', class_='highlighted', text=' Newly Enrolled Test Scores').find_next_sibling('table')
sat_scores = parse_table(test_scores)
act_scores = parse_table(test_scores.find_next_sibling('table'))

In [275]:
enrollment_table = soup.find('p', class_='highlighted', text='Degrees By Ethnicity & Gender').find_next_sibling('table')
enrollment = parse_table(enrollment_table)
enrollment = [e for e in enrollment if ('Note' not in e[0])]

In [227]:
def parse_table(table):
  return [[el.text for el in row.find_all(['th', 'td'])] for row in table.find_all('tr')]

In [300]:
base_url = 'http://profiles.asee.org'
res = loop.run_until_complete(get(base_url))
soup = BeautifulSoup(res, 'html.parser')

In [301]:
schools = soup.find('table').find_all('td', class_='school_name')
profile_links = [base_url+school.a['href'] for school in schools]

In [293]:
re.search(r'\/\d+\/', schools[0].a['href']).group(0)[1:-1]

In [310]:
res = loop.run_until_complete(get(profile_links[0]))
soup = BeautifulSoup(res, 'html.parser')
profile_ids = soup.find('select', id='survey_id').find_all('option')
profile_ids = [pid['value'] for pid in profile_ids]

In [ ]:
index = [
  'Institution Information > General Information > Type of institution'
]